In [5]:
from dotenv import load_dotenv
import os

load_dotenv("../.env")   # from notebooks/ folder


True

In [6]:
!pip install python-dotenv


In [7]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load keys from .env in Practicum1_Project1_AirQuality/
load_dotenv()
CENSUS_KEY = os.getenv("CENSUS_KEY")

print("Census Key Loaded:", CENSUS_KEY is not None)
if CENSUS_KEY is None:
    raise ValueError("❌ CENSUS_KEY is not set in your .env file")


Census Key Loaded: True


In [8]:
# Paths relative to notebooks/ folder
epa_path = "../data/epa_pm25_annual_by_county.csv"
cdc_path = "../data/cdc_asthma_by_county.csv"

pm25_df = pd.read_csv(epa_path)
asthma_df = pd.read_csv(cdc_path)

# Standardize FIPS and year types
for df in (pm25_df, asthma_df):
    df["state_fips"] = df["state_fips"].astype(str).str.zfill(2)
    df["county_fips"] = df["county_fips"].astype(str).str.zfill(3)
    df["year"] = df["year"].astype(int)

print("EPA head:")
display(pm25_df.head())

print("CDC head:")
display(asthma_df.head())

# Build target county list from EPA data (so we only pull SES for those)
target_pairs = (
    pm25_df[["state_fips", "county_fips"]]
    .drop_duplicates()
    .values
    .tolist()
)
print("Target counties (state_fips, county_fips):", target_pairs)


EPA head:


,county_name,state_fips,county_fips,year,annual_pm25
0,"Los Angeles County, CA",06,037,2018,11.924489
1,"Los Angeles County, CA",06,037,2019,10.113096
2,"Los Angeles County, CA",06,037,2020,13.435450
3,"Los Angeles County, CA",06,037,2021,12.161321
4,"Los Angeles County, CA",06,037,2022,10.929975


CDC head:


,state_fips,county_fips,year,asthma_prevalence
0,06,037,2023,9.1
1,04,013,2023,10.1
2,06,037,2023,9.0
3,04,013,2023,10.0
4,17,031,2023,9.8


Target counties (state_fips, county_fips): [['06', '037'], ['48', '201'], ['17', '031'], ['04', '013']]


In [9]:
BASE_CENSUS_URL = "https://api.census.gov/data"

def get_acs_ses_targeted(year, targets, census_key):
    """
    Pull ACS 5-year profile SES variables (income, poverty)
    for a set of (state_fips, county_fips) pairs.
    """
    frames = []

    for state_fips, county_fips in targets:
        url = f"{BASE_CENSUS_URL}/{year}/acs/acs5/profile"
        params = {
            "get": "NAME,DP03_0062E,DP03_0119PE",
            "for": f"county:{county_fips}",
            "in": f"state:{state_fips}",
            "key": census_key
        }

        r = requests.get(url, params=params)
        r.raise_for_status()
        data = r.json()

        # Single-row DataFrame for this county-year
        df = pd.DataFrame(data[1:], columns=data[0])

        df["state_fips"] = df["state"].astype(str).str.zfill(2)
        df["county_fips"] = df["county"].astype(str).str.zfill(3)
        df["year"] = year

        df["DP03_0062E"] = pd.to_numeric(df["DP03_0062E"], errors="coerce")
        df["DP03_0119PE"] = pd.to_numeric(df["DP03_0119PE"], errors="coerce")

        df = df.rename(columns={
            "DP03_0062E": "median_household_income",
            "DP03_0119PE": "poverty_percent"
        })

        frames.append(df[[
            "NAME", "state_fips", "county_fips", "year",
            "median_household_income", "poverty_percent"
        ]])

    return pd.concat(frames, ignore_index=True)


In [10]:
YEARS = list(range(2018, 2024))  # 2018, 2019, 2020, 2021, 2022, 2023

acs_frames = []
for yr in YEARS:
    print(f"Pulling ACS SES for year {yr} ...")
    acs_frames.append(get_acs_ses_targeted(yr, target_pairs, CENSUS_KEY))

acs_all = pd.concat(acs_frames, ignore_index=True)

print("ACS SES head:")
display(acs_all.head())
print("Unique state_fips in ACS:", acs_all["state_fips"].unique())


Pulling ACS SES for year 2018 ...
Pulling ACS SES for year 2019 ...
Pulling ACS SES for year 2020 ...
Pulling ACS SES for year 2021 ...
Pulling ACS SES for year 2022 ...
Pulling ACS SES for year 2023 ...
ACS SES head:


,NAME,state_fips,county_fips,year,median_household_income,poverty_percent
0,"Los Angeles County, California",06,037,2018,64251,12.2
1,"Harris County, Texas",48,201,2018,60146,13.4
2,"Cook County, Illinois",17,031,2018,62088,11.4
3,"Maricopa County, Arizona",04,013,2018,61606,10.6
4,"Los Angeles County, California",06,037,2019,68044,11.2


Unique state_fips in ACS: ['06' '48' '17' '04']


In [11]:
# Standardize FIPS again just to be safe
acs_all["state_fips"] = acs_all["state_fips"].astype(str).str.zfill(2)
acs_all["county_fips"] = acs_all["county_fips"].astype(str).str.zfill(3)
acs_all["full_fips"] = acs_all["state_fips"] + acs_all["county_fips"]

pm25_df["full_fips"] = pm25_df["state_fips"] + pm25_df["county_fips"]

print("ACS full_fips:", acs_all["full_fips"].unique())
print("EPA full_fips:", pm25_df["full_fips"].unique())


ACS full_fips: ['06037' '48201' '17031' '04013']
EPA full_fips: ['06037' '48201' '17031' '04013']


In [12]:
# Merge EPA + CDC first
epa_cdc = pm25_df.merge(
    asthma_df,
    on=["state_fips", "county_fips", "year"],
    how="left"
)

print("EPA+CDC merged head:")
display(epa_cdc.head())

# Then add ACS SES
master = epa_cdc.merge(
    acs_all[[
        "state_fips", "county_fips", "year",
        "median_household_income", "poverty_percent"
    ]],
    on=["state_fips", "county_fips", "year"],
    how="left"
)

print("Final master dataset head:")
display(master.head())
print("Columns:", master.columns.tolist())


EPA+CDC merged head:


,county_name,state_fips,county_fips,year,annual_pm25,full_fips,asthma_prevalence
0,"Los Angeles County, CA",06,037,2018,11.924489,06037,NaN
1,"Los Angeles County, CA",06,037,2019,10.113096,06037,NaN
2,"Los Angeles County, CA",06,037,2020,13.435450,06037,NaN
3,"Los Angeles County, CA",06,037,2021,12.161321,06037,NaN
4,"Los Angeles County, CA",06,037,2022,10.929975,06037,NaN


Final master dataset head:


,county_name,state_fips,county_fips,year,annual_pm25,full_fips,asthma_prevalence,median_household_income,poverty_percent
0,"Los Angeles County, CA",06,037,2018,11.924489,06037,NaN,64251,12.2
1,"Los Angeles County, CA",06,037,2019,10.113096,06037,NaN,68044,11.2
2,"Los Angeles County, CA",06,037,2020,13.435450,06037,NaN,71358,10.5
3,"Los Angeles County, CA",06,037,2021,12.161321,06037,NaN,76367,10.2
4,"Los Angeles County, CA",06,037,2022,10.929975,06037,NaN,83411,10.0


Columns: ['county_name', 'state_fips', 'county_fips', 'year', 'annual_pm25', 'full_fips', 'asthma_prevalence', 'median_household_income', 'poverty_percent']


In [13]:
output_path = "../data/air_quality_health_ses_merged.csv"
master.to_csv(output_path, index=False)

print("✅ FINAL MASTER DATASET SAVED TO:", output_path)
print("Shape:", master.shape)


✅ FINAL MASTER DATASET SAVED TO: ../data/air_quality_health_ses_merged.csv
Shape: (28, 9)


In [14]:
master_clean = (
    master
    .sort_values(by=["state_fips", "county_fips", "year"])
    .drop_duplicates(
        subset=["state_fips", "county_fips", "year"],
        keep="mean" if False else "first"
    )
)

master_clean.to_csv("../data/air_quality_health_ses_merged_clean.csv", index=False)

print("✅ Cleaned master dataset saved")
print(master_clean.shape)


✅ Cleaned master dataset saved
(24, 9)
